In [ ]:
# To avoid certain errors that occur in some systems due to conflicts of paths.
import os, sys, pprint
# Add the parent directory of export to the Python path
working_directory = os.getcwd()
parent_dir = os.path.join(os.getcwd(), os.pardir)
p_parent_dir = os.path.join(parent_dir, os.pardir) 
sys.path.append(os.path.abspath(p_parent_dir))

## Environment and Imports
The python working environment required to run this jupyternotebook can be cloned from /templates/environment.yml. Then, run the following import block.

In [ ]:
#All the import statements required in the generation of lci inventory as per the LCIS
import bw2data as bw
import bw2io as bwio
import shutil as sh
#import json
import pandas as pd
import export.libs.metadata as md
import export.libs.excel as excel

## Initialize

Run this cell to initalize the project, database and other  variables required for the export.

In [ ]:
#project and database names
project_name = input("Enter the name of the project: ")
database_name = input("Enter the name of the database: ")
bw.projects.set_current(project_name)
selected_database = bw.Database(database_name)

# All output files will contain the exported_file_name as the prefix
exported_file_name = input("Enter the desired name for the export files: ")

#LCI schema and dataset version number
schema_version = input ("Enter the Schema version (e.g. LCIS2024): ")
dataset_version = input ("Enter the version of the Dataset (e.g 1.0.1): ")

## Dataset

### csv Inventory file

In [ ]:
print ('The current working directory is ', working_directory)
# Exports the csv inventory file
csv_file_location = bwio.export.csv.write_lci_csv(database_name) #this excludes nested data
sh.copy(csv_file_location, exported_file_name+'.csv' )

### xlsx Inventory file

In [ ]:
# Exports the xlsx inventory file and improves the formatting
input_excel_file_location = bwio.export.excel.write_lci_excel(database_name)
output_excel_file = input_excel_file_location + '.xlsx'
excel.excel_format(input_excel_file_location, output_excel_file)
sh.copy(output_excel_file, exported_file_name + '.xlsx' )

## Dataset properties

General properties of the database such as total no. of activities, exchanges, project parameters etc. At present, the database versioning has to performed manually. Methods of making this process a bit intelligent (Depending on the underlying changes in the data) has to be investigated separately.

In [ ]:
identifying_properties = md.db_props(project_name, database_name).identifying_info(schema_version, dataset_version)
general_properties = md.db_props(project_name, database_name).gen_info()
database_properties = identifying_properties | general_properties
# fetch the names of dependent databases within the selcted database. Append this to the database properties dict.
database_properties ["Dataset dependencies"] = md.db_props(project_name, database_name).db_dependencies()
pprint.pprint(database_properties)

Write database_properties to a csv file

In [ ]:
db_props_csv_file = exported_file_name + '_properties.csv'
md.db_props.csv(db_props_csv_file, database_properties)

## Metadata

The metadata excel template is available at /export/templates/. Once the the releveant metadata has been updated, conversion to csv can be done in Office suit or using the conversion function below.

In [ ]:
input_file_path = input("Enter the path to the Excel metadata file: ") # Ask the user to provide the Excel file path
output_metadata_csv_file = exported_file_name + '_metadata.csv'
md.metadata().excel_to_csv(input_file_path, output_metadata_csv_file)

# Depreciated

In [ ]:
file_location = bwio.export.csv.write_lci_csv(database_name) #this excludes nested data
working_directory = os.getcwd()
print ('The current working directory is ', working_directory)
sh.copy(file_location, exported_file_name+'.csv' )

In [ ]:
excel_file_location = bwio.export.excel.write_lci_excel(database_name)
working_directory = os.getcwd()
print ('The current working directory is ', working_directory)
sh.copy(excel_file_location, exported_file_name + '.xlsx' )

Format the above-generated excel file to improve human readabiliy

In [ ]:
excel_file = input("Enter the path of your excel file: ")
output_excel_file = exported_file_name + '_formatted.xlsx'
excel.excel_format(excel_file, output_excel_file)

In [ ]:
database_properties = md.db_props(project_name, database_name).gen_info()
import uuid
uid = uuid.uuid4().hex
database_properties['Database Identifier'] = uid
database_properties['Version'] = input ("Enter the version of the database (e.g 1.0.1): ")
pprint.pprint (database_properties)

In [ ]:
lci_db = lci.LCI(project_name, database_name)

#declare the important info
keys_technosphere = ['name', 'database', 'location', 'unit','reference product']
keys_biosphere = ['name', 'database', 'unit', 'categories']
keys_production = ['name', 'database', 'location', 'unit','reference product']

# the life cycle inventory is stored as a list of dicts with first element of list containing parameters and second element containing activity dics.
LCI_list = lci_db.LCI_database_list(keys_biosphere, keys_technosphere, keys_production)
# pprint.pprint(LCI_list, indent=2)

In [ ]:
dependencies = md.db_props.db_dependencies(LCI_list[1])
database_properties['Dataset Dependencies'] = dependencies
pprint.pprint (database_properties)

In [ ]:
input_file_path = input("Enter the path to the Excel metadata file: ")         # Ask the user to provide the Excel file path
output_metadata_file = exported_file_name + '_metadata.md'

md.metadata().excel_to_md(input_file_path, output_metadata_file)